## RAG system using Llama2 with Huggingface



In [ ]:
!pip install pypdf --quiet

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes --quiet
#bitsandbytes - for quantization, accelerate - speedup of uploading

In [ ]:
## Embedding
!pip install sentence_transformers --quiet

In [ ]:
!pip install llama-index-embeddings-huggingface --quiet

In [ ]:
!pip install llama_index --quiet
!pip install llama-index-llms-huggingface --quiet

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
#from llama_index.prompts.base import SimpleInputPrompt
from llama_index.core import PromptTemplate

In [ ]:
documents=SimpleDirectoryReader("/content/data").load_data()
documents

[Document(id_='068f30eb-a6cb-4616-8ae4-e1585d53fd18', embedding=None, metadata={'page_label': '1', 'file_name': 'Invoice_691813.pdf', 'file_path': '/content/data/Invoice_691813.pdf', 'file_type': 'application/pdf', 'file_size': 38712, 'creation_date': '2024-10-22', 'last_modified_date': '2024-10-22'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='INVOICE\nFrom\nLearnAI\nInnovation 4.0\n3 Research Link #02-04\nSingapore\n117602\nBill to\nMishra Gour Chandra\ngourc.mishra@gmail.com\n91697840Invoice no:  691813\nOrder date:  06-10-2022\nS.No Product Quantity Unit price Total price\n1 Free Membership 1 $0.00 $0.00\nSubtotal $0.00  (incl. tax)\nTotal $0.00', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{con

In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
# Define query_wrapper_prompt here
query_wrapper_prompt = PromptTemplate(
    "### Instruction:\n{query_str}\n\n### Response:\n",
    system_prompt=system_prompt,
)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGr

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    #model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model


In [ ]:
# a vector store index only needs an embed model
index = VectorStoreIndex.from_documents(
    documents, embed_model=embed_model
)
# ... until you create a query engine
query_engine = index.as_query_engine()

In [ ]:
response=query_engine.query("Summarize the content")
print(response)
